# Obdelava podatkov

In [20]:
import re
import requests
import os
import csv

## 1. Pridobivanje podatkov
Prvo definiramo spremenljivke:

In [21]:
podatki_url = 'https://en.wikipedia.org/wiki/List_of_chemical_elements'

directory_dat = 'Surovi podatki'

datoteka_html = 'podatki.html'

elementi = 'elementi.csv'


Potem napišemo funkcijo, ki podatke dobi iz spleta:

In [22]:
def nalozi_iz_spleta(url):
    
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.text
        else:
            raise ValueError(f'Prišlo je do čudne kode: {response.status_code}')
    except Exception:
        print(f'Prišlo je do napake pri nalaganju strani {url}')

potem napisemo funkcijo, ki bo te podatke shranila v file:

In [23]:
def shrani_v_datoteko(text, directory, file):
    os.makedirs(directory, exist_ok=True)
    path = os.path.join(directory, file)
    with open(path, 'w', encoding='utf-8') as f:
        f.write(text)
    return None

in še funkcije, ki združi prejšnji funkciji:

In [24]:
def shrani(page, directory, file):
    vsebina = nalozi_iz_spleta(page)
    if vsebina != None:
        shrani_v_datoteko(vsebina, directory, file) 

Potem poženemo funkcijo, ki nam iz spletne strani shrani html kodo v datoteko na računalniku.

In [25]:
shrani(podatki_url, directory_dat, datoteka_html)

## 2. Izluščenje podatkov

Napišemo funkcijo, ki bo prebrala datoteko.

In [26]:
def preberi_datoteko(dir, dat):
    pat = os.path.join(dir, dat)
    with open(pat, 'r', encoding = 'utf-8') as f:
        return f.read()

Potem napišemo funkcijo, ki nam bo iz tega izluščila zaželjene elemente.

In [27]:
def luscenje(dir, dat):
    #najprej iz html kode izoliramo tabelo
    surovo = preberi_datoteko(dir, dat)
    zac_tab = surovo.index(r'<table')
    kon_tab = surovo.index(r'</table')
    tabela = surovo[zac_tab : kon_tab]
    vrstice = re.split('<tr', tabela) #naredimo seznam vrstic
    vrne = []
    for vrstica in vrstice[5:]:     #spustimo glavo
        celice = re.split(r'<td.*?>', vrstica, flags=re.DOTALL)     #ločimo vrstico na celice
        vsebina_vr = []
        for x in celice[1:]:
            izl_celica = re.sub(r'<.*?>', '', x)
            izl_celica = re.sub(r'\n*', '', izl_celica)
            izl_celica = re.sub(r'&#.*;', '', izl_celica) # pri nekaterih mankajocih podathih je izluscilo to kodo namesto –
            izl_celica = re.sub('–', '', izl_celica)
            vsebina_vr.append(izl_celica)
        vrne.append({
            'atomsko_st' : vsebina_vr[0],
            'simbol' : vsebina_vr[1],
            'ime' : vsebina_vr[2],
            'izvor_imena' : vsebina_vr[3],
            'skupina' : vsebina_vr[4],
            'perioda' : vsebina_vr[5],
            'blok' : vsebina_vr[6],
            'rel_at_masa' : vsebina_vr[7],
            'gostota' : vsebina_vr[8],
            'talisce' : vsebina_vr[9],
            'vrelisce' : vsebina_vr[10],
            'spec_toplota' : vsebina_vr[11],
            'el_neg' : vsebina_vr[12],
            'pogostost_v_zem_skorji' : vsebina_vr[13],
            'izvor' : vsebina_vr[14],
            'argegato' : vsebina_vr[15]
            })
    return vrne


In preverimo, če deluje.

In [28]:
p = luscenje(directory_dat, datoteka_html)
print(p)

[{'atomsko_st': '1', 'simbol': 'H', 'ime': 'Hydrogen', 'izvor_imena': "Greek elements hydro- and -gen, 'water-forming'", 'skupina': '1', 'perioda': '1', 'blok': 's-block', 'rel_at_masa': '1.0080', 'gostota': '0.00008988', 'talisce': '14.01', 'vrelisce': '20.28', 'spec_toplota': '14.304', 'el_neg': '2.20', 'pogostost_v_zem_skorji': '1400', 'izvor': 'primordial', 'argegato': 'gas'}, {'atomsko_st': '2', 'simbol': 'He', 'ime': 'Helium', 'izvor_imena': "Greek hḗlios, 'sun'", 'skupina': '18', 'perioda': '1', 'blok': 's-block', 'rel_at_masa': '4.0026', 'gostota': '0.0001785', 'talisce': '', 'vrelisce': '4.22', 'spec_toplota': '5.193', 'el_neg': '', 'pogostost_v_zem_skorji': '0.008', 'izvor': 'primordial', 'argegato': 'gas'}, {'atomsko_st': '3', 'simbol': 'Li', 'ime': 'Lithium', 'izvor_imena': "Greek líthos, 'stone'", 'skupina': '1', 'perioda': '2', 'blok': 's-block', 'rel_at_masa': '6.94', 'gostota': '0.534', 'talisce': '453.69', 'vrelisce': '1560', 'spec_toplota': '3.582', 'el_neg': '0.98', 

## 3. Shranjevanje podatko

Napišemo funkcijo, ki nam bo zgornje izlušene podatke shranila v csv datoteko.

In [29]:
def zapisi_v_csv(dat, csv_file):
    stolpci = ['atomsko_st', 'simbol', 'ime', 'izvor_imena', 'skupina', 'perioda', 'blok', 'rel_at_masa', 'gostota', 'talisce', 'vrelisce', 'spec_toplota', 'el_neg', 'pogostost_v_zem_skorji', 'izvor', 'argegato']
    with open(csv_file, 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=stolpci)
        writer.writeheader()
        writer.writerows(dat)
    print('Uspesno zapisano v', csv_file)

In [30]:
zapisi_v_csv(p, elementi)

Uspesno zapisano v elementi.csv
